In [1]:
from netCDF4 import Dataset, num2date
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
#gives the postion of a particular date in a particular array. 
#However, if you want 1974 Jan 13, date should be datetime.datetime(1974, 12, 13, 0, 0)
def MISOdate_position(date):
    pos = np.where(date_miso==date)
    return pos[0][0]

def OLRdate_position(date):
    pos = np.where(date_olr==date)
    return pos[0][0]

#gives the position of a particular [latitude, longutude] in arrays of latitude and longitudes
#latitude ranges from (90 to -90 with step = -2.5)
#longigute ranges from (0 to 357.5 with step = 2.5)
def latlonpos(coordinate):
    latitude = np.where(lats==coordinate[0])[0][0]
    longitude = np.where(lons==coordinate[1])[0][0]
    return([latitude, longitude])

#calculates the miso amplitude given miso1 and miso2 as defined in the literature and can be
#accessed at https://link.springer.com/content/pdf/10.1007%2Fs00382-016-3491-y.pdf
def MISOAmplitude(miso_one, miso_two):
    miso1_var = np.var(miso1)
    miso2_var = np.var(miso2)
    amp = np.sqrt(((miso_one*miso_one)/miso1_var) + ((miso_two * miso_two)/miso2_var))
    return (amp)

#computes the miso phase give miso1 and miso2 as defined in the literature 
def MISOPhase(miso_one, miso_two):
    if MISOAmplitude(miso_one, miso_two)<1.5:
        return (0)
    elif miso_one >0 and miso_two<0:
        if abs(miso_two)>abs(miso_one):
            return (1)
        if abs(miso_two)<abs(miso_one):
            return(2)
    elif miso_one >0 and miso_two>0:
        if abs(miso_two)<abs(miso_one):
            return(3)
        if abs(miso_two)>abs(miso_one):
            return(4)
    elif (miso_two)>0 and (miso_one)<0:
        if abs(miso_two)>abs(miso_one):
            return (5)
        if abs(miso_two)<abs(miso_one):
            return(6)
    elif (miso_two)<0 and (miso_one)<0:
        if abs(miso_one)>abs(miso_two):
            return (7)
        if abs(miso_one)<abs(miso_two):
            return (8)
            

In [3]:
    

#MISO Dataset provided by Dr. Ajaya Ravindran
miso_datasets = Dataset('miso.nc', mode = 'r')
miso1 = np.array(miso_datasets.variables['miso1'][: ])
miso2 = np.array(miso_datasets.variables['miso2'][:])
time_miso = np.array(miso_datasets.variables['time'][:]) #units: days since 1998-02-02 00:00
time_miso_unit = miso_datasets.variables['time'].units #units: days since 1998-02-02 00:00
date_miso = num2date(time_miso, units = time_miso_unit, calendar = 'standard') #assuming calendar is standard as the information is not present





#Daily Mean OLR Dataset downloaded from 
#https://www.esrl.noaa.gov/psd/data/gridded/data.interp_OLR.html

olr_datasets=Dataset('olr.day.mean.nc')
lons = np.array(olr_datasets.variables['lon'][:])
lats = np.array(olr_datasets.variables['lat'][:])
time_olr = np.array(olr_datasets.variables['time'][:])
time_olr_unit = olr_datasets.variables['time'].units
date_olr = num2date(time_olr, units = time_olr_unit, calendar = 'standard') #assuming that the calendar is standard as the information is not provided
olr = np.array(olr_datasets.variables['olr'][:]) #(time, column (latitude), row(longitude))

In [4]:
#Calculating miso_phase for all dates
miso_phase = []
miso_amp = []
for i in time_miso:
    ph = MISOPhase(miso1[i], miso2[i])
    amp = MISOAmplitude(miso1[i], miso2[i])
    miso_phase.append(ph)
    miso_amp.append(amp)
miso_phase = np.array(miso_phase)
miso_amp = np.array(miso_amp)

In [5]:
#Flattening OLR data
olr_flattened = []
for index in range(len(olr)):
    temp = olr[index].flatten()
    olr_flattened.append(temp)

In [6]:
#Creating dataframe using miso1, miso2, phase, amplitude and time(days since 1998)
temp = {'MISO1': miso1, 'MISO2':miso2, 'Phase': miso_phase, 'Amplitude': miso_amp}
miso_dataframe = pd.DataFrame(data=temp, index = date_miso)


#Creating dataframe using flattened points (olr) and date
temp = {'Flattened_olr': olr_flattened}
olr_dataframe = pd.DataFrame(data= temp, index = date_olr)

In [7]:
#Merging two dataframe
final_dataframe=olr_dataframe.merge(miso_dataframe,how='inner', left_index=True, right_index=True)

In [8]:
#Saving dataframe
final_dataframe.to_pickle('final_dataframe.pkl')